# Chargement de dossiers multi-canaux / multi-positions de fichiers tif
Certains microscopes écrivent les données d'image sur le disque sous forme de fichiers tif, tranche par tranche. Vous trouverez alors de nombreux fichiers nommés par exemple `image_z03_ch01.tif` dans ces dossiers. Pour charger ces dossiers, vous devez généralement connaître des détails tels que le nombre de tranches z et le nombre de canaux imagés, ou si les données d'image ont plusieurs points temporels ou non.

Voir aussi [cette discussion](https://forum.image.sc/t/reading-2d-images-as-a-multi-channel-3d-stack-in-python/75460).

In [1]:
from skimage import io
import matplotlib.pyplot as plt
import tifffile as tif
import numpy as np

import shutil
import os

path = "../../data/tif_folder/"

À des fins de démonstration, nous créons simplement un tel dossier avec des données d'image factices.

In [2]:
if not os.path.exists(path):
    os.mkdir(path)

In [3]:
for z in range(1,7):
    for c in range(1,5):
        #image = io.imread("c:/structure/data/blobs.tif")
        #io.imsave(f"c:/structure/data/images/r01c01f34p0{z}-ch0{c}t01.tiff", image)
        
        shutil.copy("../../data/blobs.tif",
                    path + f"image_z{str(z).zfill(2)}-ch{str(c).zfill(2)}.tiff")

Pour avoir un aperçu, nous pouvons afficher les noms de fichiers dans le dossier.

In [4]:
for file in os.listdir(path):
    print(file)

image_z01-ch01.tiff
image_z01-ch02.tiff
image_z01-ch03.tiff
image_z01-ch04.tiff
image_z02-ch01.tiff
image_z02-ch02.tiff
image_z02-ch03.tiff
image_z02-ch04.tiff
image_z03-ch01.tiff
image_z03-ch02.tiff
image_z03-ch03.tiff
image_z03-ch04.tiff
image_z04-ch01.tiff
image_z04-ch02.tiff
image_z04-ch03.tiff
image_z04-ch04.tiff
image_z05-ch01.tiff
image_z05-ch02.tiff
image_z05-ch03.tiff
image_z05-ch04.tiff
image_z06-ch01.tiff
image_z06-ch02.tiff
image_z06-ch03.tiff
image_z06-ch04.tiff


scikit-image propose une fonction [`imread_collection`](https://scikit-image.org/docs/stable/api/skimage.io.html#skimage.io.imread_collection) pour charger des fichiers correspondant à un motif, par exemple contenant `*`.

In [5]:
im_collection = io.imread_collection(path + "*")
im_collection

Vous pouvez transformer cette collection d'images en une pile d'images basée sur un tableau numpy. Malheureusement, le nombre de tranches z et de canaux est inconnu à ce stade.

In [6]:
image_stack = im_collection.concatenate()
image_stack.shape

(24, 254, 256)

Si vous connaissez le nombre de tranches z et de canaux, vous pouvez [`reshape`]() l'image en une image 3D+ch ou 4D.

In [7]:
num_channels = 4
num_z_slices = 6
image4d = np.reshape(image_stack, (num_channels, num_z_slices, image_stack.shape[-2], image_stack.shape[-1]))
image4d.shape

(4, 6, 254, 256)

Alternativement, vous pouvez également construire vos propres boucles for pour charger les données à partir du disque. Cela vous donne un peu plus de liberté, par exemple pour trier les tranches et les canaux dans les dimensions utilisées.

In [8]:
num_channels = 4
num_z_slices = 6

image4d_loaded = np.asarray([
    [io.imread(path + f"image_z{str(z).zfill(2)}-ch{str(c).zfill(2)}.tiff") for c in range(1, 5)]
    for z in range(1, 7)
])
image4d_loaded.shape

(6, 4, 254, 256)